In [24]:
import scipy.io as sio
import numpy as np
import tqdm
import math
import pickle

In [7]:
# coordinate of 32k L-sphere, R-sphere
L_coord32k_sphere = sio.loadmat("data/Sphere_L_32k.mat")["Lcoordinate"]
R_coord32k_sphere = sio.loadmat("data/Sphere_R_32k.mat")["Rcoordinate"]

# load data storing which of the 32k coordinate we want to use
L_32k_to_29k = sio.loadmat("data/32k_to_29k.mat")["tem5"]
R_32k_to_29k = sio.loadmat("data/R_32k_to_29k.mat")["tem11"]

In [10]:
# only use 29k
# 32K to 29K
L_coord29k_sphere = L_coord32k_sphere[np.all(L_32k_to_29k == 1, axis = 1)]

R_coord29k_sphere = R_coord32k_sphere[np.all(R_32k_to_29k == 1, axis = 1)]

In [14]:
# make a dict that corresponds a coordinate in the atlas to that of an icosahedron

phi = (1+ math.sqrt(5))/2
scale = 100 / math.sqrt(1+phi**2)
atlas_to_coordinate = {}

atlas_to_coordinate[(0,0,0)] = scale * np.array([-1,0,phi])
atlas_to_coordinate[(0,64,0)] = scale * np.array([-phi, -1,0])
atlas_to_coordinate[(0,128,0)] = scale * np.array([-1,0,-phi])
atlas_to_coordinate[(0,0,64)] = scale * np.array([1,0,phi])
atlas_to_coordinate[(0,64,64)] = scale * np.array([0,-phi,1])
atlas_to_coordinate[(0,128,64)] = scale * np.array([0,-phi,-1])

atlas_to_coordinate[(1,0,0)] = scale * np.array([0,phi,1])
atlas_to_coordinate[(1,64,0)] = scale * np.array([-phi,1,0])
atlas_to_coordinate[(1,128,0)] = scale * np.array([-1,0,-phi])
atlas_to_coordinate[(1,0,64)] = scale * np.array([1,0,phi])
atlas_to_coordinate[(1,64,64)] = scale * np.array([-1,0,phi])
atlas_to_coordinate[(1,128,64)] = scale * np.array([-phi, -1,0])

atlas_to_coordinate[(2,0,0)] = scale * np.array([phi,1,0])
atlas_to_coordinate[(2,64,0)] = scale * np.array([0,phi,-1])
atlas_to_coordinate[(2,128,0)] = scale * np.array([-1,0,-phi])
atlas_to_coordinate[(2,0,64)] = scale * np.array([1,0,phi])
atlas_to_coordinate[(2,64,64)] = scale * np.array([0,phi,1])
atlas_to_coordinate[(2,128,64)] = scale * np.array([-phi,1,0])

atlas_to_coordinate[(3,0,0)] = scale * np.array([phi,-1,0])
atlas_to_coordinate[(3,64,0)] = scale * np.array([1,0,-phi])
atlas_to_coordinate[(3,128,0)] = scale * np.array([-1,0,-phi])
atlas_to_coordinate[(3,0,64)] = scale * np.array([1,0,phi])
atlas_to_coordinate[(3,64,64)] = scale * np.array([phi,1,0])
atlas_to_coordinate[(3,128,64)] = scale * np.array([0,phi,-1])

atlas_to_coordinate[(4,0,0)] = scale * np.array([0,-phi,1])
atlas_to_coordinate[(4,64,0)] = scale * np.array([0,-phi,-1])
atlas_to_coordinate[(4,128,0)] = scale * np.array([-1,0,-phi])
atlas_to_coordinate[(4,0,64)] = scale * np.array([1,0,phi])
atlas_to_coordinate[(4,64,64)] = scale * np.array([phi,-1,0])
atlas_to_coordinate[(4,128,64)] = scale * np.array([1,0,-phi])

In [15]:
# make a dict of triangles; the key is the depth of the icosahedron

triangles = {}

triangles[0] = []

triangles[0].append(((0,0,0),(0,0,64),(0,64,64)))
triangles[0].append(((0,0,0),(0,64,0),(0,64,64)))
triangles[0].append(((0,128,64),(0,64,0),(0,64,64)))
triangles[0].append(((0,64,0),(0,128,0),(0,128,64)))

triangles[0].append(((1,0,0),(1,0,64),(1,64,64)))
triangles[0].append(((1,0,0),(1,64,0),(1,64,64)))
triangles[0].append(((1,128,64),(1,64,0),(1,64,64)))
triangles[0].append(((1,64,0),(1,128,0),(1,128,64)))

triangles[0].append(((2,0,0),(2,0,64),(2,64,64)))
triangles[0].append(((2,0,0),(2,64,0),(2,64,64)))
triangles[0].append(((2,128,64),(2,64,0),(2,64,64)))
triangles[0].append(((2,64,0),(2,128,0),(2,128,64)))

triangles[0].append(((3,0,0),(3,0,64),(3,64,64)))
triangles[0].append(((3,0,0),(3,64,0),(3,64,64)))
triangles[0].append(((3,128,64),(3,64,0),(3,64,64)))
triangles[0].append(((3,64,0),(3,128,0),(3,128,64)))

triangles[0].append(((4,0,0),(4,0,64),(4,64,64)))
triangles[0].append(((4,0,0),(4,64,0),(4,64,64)))
triangles[0].append(((4,128,64),(4,64,0),(4,64,64)))
triangles[0].append(((4,64,0),(4,128,0),(4,128,64)))

In [16]:
# make new triangles while storing data into make_triangles and atlas_to_coordinate

def make_triangles(coord0, coord1, coord2, depth):
    
    atlas_to_coordinate[(coord0[0], int((coord1[1]+coord0[1])/2), int((coord1[2]+coord0[2])/2))] = (atlas_to_coordinate[coord0]+atlas_to_coordinate[coord1])/2
                         
    atlas_to_coordinate[(coord0[0], int((coord2[1]+coord1[1])/2), int((coord2[2]+coord1[2])/2))] = (atlas_to_coordinate[coord2]+atlas_to_coordinate[coord1])/2
                         
    atlas_to_coordinate[(coord0[0], int((coord2[1]+coord0[1])/2), int((coord2[2]+coord0[2])/2))] = (atlas_to_coordinate[coord0]+atlas_to_coordinate[coord2])/2
    
    triangles[depth].append((
        coord0, 
        (coord0[0], int((coord0[1]+coord1[1])/2), int((coord0[2]+coord1[2])/2)), 
        (coord0[0], int((coord2[1]+coord0[1])/2), int((coord2[2]+coord0[2])/2))
    ))     
    
    triangles[depth].append((
        coord1, 
        (coord0[0], int((coord0[1]+coord1[1])/2), int((coord0[2]+coord1[2])/2)), 
        (coord0[0], int((coord2[1]+coord1[1])/2), int((coord2[2]+coord1[2])/2))
    ))   
    
    triangles[depth].append((
        coord2, 
        (coord0[0], int((coord2[1]+coord1[1])/2), int((coord2[2]+coord1[2])/2)), 
        (coord0[0], int((coord2[1]+coord0[1])/2), int((coord2[2]+coord0[2])/2))
    ))   
    
    triangles[depth].append((
        (coord0[0], int((coord2[1]+coord1[1])/2), int((coord2[2]+coord1[2])/2)), 
        (coord0[0], int((coord0[1]+coord1[1])/2), int((coord0[2]+coord1[2])/2)), 
        (coord0[0], int((coord2[1]+coord0[1])/2), int((coord2[2]+coord0[2])/2))
    ))

In [17]:
# make three new triangles for each triangle

def dig_deeper(current_depth):
    
    triangles[current_depth + 1] = []
    
    for triangle in triangles[current_depth]:
        make_triangles(triangle[0], triangle[1], triangle[2], current_depth + 1)

In [18]:
# For our purpose, we want to do this for six times

for i in range(6):
    dig_deeper(i)

In [23]:
# atlas_to_coordinate, triangles

In [ ]:
# for each point in the atlas, store info about the three nearest points from 29k
# (If you were to run in the future, using distance matrix might be a faster algorithm (takes about 4 hours to run

for atlas in tqdm.tqdm(atlas_to_coordinate.keys()):

    coordinates = atlas_to_coordinate[atlas]

    L_min_three = []

    R_min_three = []

    for i in range(L_coord29k_sphere.shape[0]):

        # distance between Left coordinate and the icosahedron
        distance = LA.norm(coordinates - L_coord29k_sphere[i])

        if len(L_min_three) < 3:
            L_min_three.append( (distance, i) )

        else:

            if max(L_min_three)[0] > distance:

                L_min_three.remove(max(L_min_three))

                L_min_three.append( (distance, i) )

    
    for i in range(R_coord29k_sphere.shape[0]):

        # distance between Right coordinate and the icosahedron        
        distance = LA.norm(coordinates - R_coord29k_sphere[i])

        if len(R_min_three) < 3:

            R_min_three.append( (distance, i) )


        else:

            if max(R_min_three)[0] > distance:

                R_min_three.remove(max(R_min_three))

                R_min_three.append( (distance, i) )


    atlas_to_coordinate[atlas] = np.append(coordinates, L_min_three + R_min_three)

In [ ]:
with open('atlas_to_coordinate.pickle', 'wb') as f:
    
    pickle.dump(atlas_to_coordinate, f)
    
with open('atlas_to_coordinate.pickle', 'rb') as f:
    
    saved_atlas_to_coordinate = pickle.load(f)